In [1]:
# https://www.kaggle.com/code/sarmat/lgbm-stacking-example/notebook

In [1]:
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from dataset import custom_train_test_split, make_dataset

def get_metric(targets, preds):
    auc = roc_auc_score(targets, preds)
    acc = accuracy_score(targets, np.where(preds >= 0.5, 1, 0))
    precsion = precision_score(targets, np.where(preds >= 0.5, 1, 0))
    recall = recall_score(targets, np.where(preds >= 0.5, 1, 0))
    F1_score = f1_score(targets, np.where(preds >= 0.5, 1, 0))

    return auc, acc, precsion, recall, F1_score

In [2]:
device = 'cuda'
param1 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 44,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.12
            
            }

param2 = {   
            'boosting': 'gbdt', 
            'objective': 'binary',
            'num_leaves' : 46,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.11
            }

param3 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 48,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.10
            }

param4 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 50,
            'metric' : 'auc',
            'learning_rate' : 0.09
            
            }

param5 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 52,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.08
            }


In [3]:
cat_cols = ['i_head', 'i_mid','i_tail', 'hour', 'dow']
cont_cols = [                        
        'user_correct_answer',
        'user_total_answer',
        'user_acc',            
        't_elapsed',            
        'cum_correct',
        'last_problem',
        'head_term',
        # 'left_asymptote',
        'elo_prob',
        'pkt',
        'u_head_mean',
        'u_head_count',
        'u_head_std',
        'u_head_elapsed',
        'i_mid_elapsed',
        'i_mid_mean',
        'i_mid_std',
        'i_mid_sum',
        'i_mid_count',
        'i_mid_tag_count',
        'assessment_mean',
        'assessment_sum',
        # 'assessment_std',
        'tag_mean',
        'tag_sum',
        # 'tag_std',
        'tail_mean',
        'tail_sum',
        # 'tail_std',
        'hour_mean',
        'hour_sum',
        # 'hour_std',
        'dow_mean',
        'dow_sum',
        # 'dow_std',
        'tag_elapsed',
        'tag_elapsed_o',
        'tag_elapsed_x',
        'assessment_elapsed',
        'assessment_elapsed_o',
        'assessment_elapsed_x',
        'tail_elapsed',
        'tail_elapsed_o',
        'tail_elapsed_x']

FEATS = cat_cols + cont_cols

In [4]:
train = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/all.pkl')
y = train.answerCode
# train, test = custom_train_test_split(train_data)
# y_train, train, y_test, test = make_dataset(train, test)
test = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/test_data.pkl')
test = test[test.answerCode==-1]

In [5]:
kf = KFold(n_splits=3, random_state = 42,shuffle = True)

oof_clf1 = np.zeros(len(train))
oof_clf2 = np.zeros(len(train))
oof_clf3 = np.zeros(len(train))
oof_clf4 = np.zeros(len(train))
oof_clf5 = np.zeros(len(train))

oof_clf1_t = np.zeros(len(test))
oof_clf2_t = np.zeros(len(test))
oof_clf3_t = np.zeros(len(test))
oof_clf4_t = np.zeros(len(test))
oof_clf5_t = np.zeros(len(test))

for (trn_idx, val_idx),test_val_idx in zip(kf.split(train.values, y.values),kf.split(test.values)):
        test_val_idx = test_val_idx[0]
        
        lgb_train = lgb.Dataset(train[FEATS].iloc[trn_idx], label = y.iloc[trn_idx]) 
        lgb_valid = lgb.Dataset(train[FEATS].iloc[val_idx], label = y.iloc[val_idx]) 
        
        model1 = lgb.train(param1, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model2 = lgb.train(param2, lgb_train, 10000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model3 = lgb.train(param3, lgb_train, 10000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model4 = lgb.train(param4, lgb_train, 10000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model5 = lgb.train(param5, lgb_train, 10000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)            
        
        oof_clf1[val_idx] = model1.predict(train[FEATS].iloc[val_idx])
        oof_clf2[val_idx] = model2.predict(train[FEATS].iloc[val_idx])
        oof_clf3[val_idx] = model3.predict(train[FEATS].iloc[val_idx])
        oof_clf4[val_idx] = model4.predict(train[FEATS].iloc[val_idx])
        oof_clf5[val_idx] = model5.predict(train[FEATS].iloc[val_idx])
        
        oof_clf1_t[test_val_idx] = model1.predict(test[FEATS].iloc[test_val_idx])
        oof_clf2_t[test_val_idx] = model2.predict(test[FEATS].iloc[test_val_idx])
        oof_clf3_t[test_val_idx] = model3.predict(test[FEATS].iloc[test_val_idx])
        oof_clf4_t[test_val_idx] = model4.predict(test[FEATS].iloc[test_val_idx])
        oof_clf5_t[test_val_idx] = model5.predict(test[FEATS].iloc[test_val_idx])
        
        # print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf1[val_idx])))
        print("AUC LGB2:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf2[val_idx])))
        print("AUC LGB3:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf3[val_idx])))
        print("AUC LGB4:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf4[val_idx])))
        print("AUC LGB5:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf5[val_idx]))) 

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['dow', 'hour', 'i_head', 'i_mid', 'i_tail']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1102675, number of negative: 581295
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041934 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6623
[LightGBM] [Info] Number of data points in the train set: 1683970, number of used features: 43


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654807 -> initscore=0.640236
[LightGBM] [Info] Start training from score 0.640236
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.428653
[200]	valid_0's binary_logloss: 0.427123
[300]	valid_0's binary_logloss: 0.426917
[400]	valid_0's binary_logloss: 0.426658
[500]	valid_0's binary_logloss: 0.426563
Early stopping, best iteration is:
[481]	valid_0's binary_logloss: 0.426511
[LightGBM] [Info] Number of positive: 1102675, number of negative: 581295
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037941 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6623
[LightGBM] [Info] Number of data points in the train set: 1683970, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654807 -> initscore=0.640236
[LightGBM] [In

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['dow', 'hour', 'i_head', 'i_mid', 'i_tail']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654337 -> initscore=0.638159
[LightGBM] [Info] Start training from score 0.638159
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.427742
[200]	valid_0's binary_logloss: 0.426465
[300]	valid_0's binary_logloss: 0.426091
[400]	valid_0's binary_logloss: 0.426032
[500]	valid_0's binary_logloss: 0.426026
Early stopping, best iteration is:
[433]	valid_0's binary_logloss: 0.425933
[LightGBM] [Info] Number of positive: 1101885, number of negative: 582086
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044711 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6622
[LightGBM] [Info] Number of data points in the train set: 1683971, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654337 -> initscore=0.638159
[LightGBM] [In

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['dow', 'hour', 'i_head', 'i_mid', 'i_tail']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1102616, number of negative: 581355
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037098 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6623
[LightGBM] [Info] Number of data points in the train set: 1683971, number of used features: 43


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654771 -> initscore=0.640079
[LightGBM] [Info] Start training from score 0.640079
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.428142
[200]	valid_0's binary_logloss: 0.426663
[300]	valid_0's binary_logloss: 0.426313
[400]	valid_0's binary_logloss: 0.426148
[500]	valid_0's binary_logloss: 0.426117
Early stopping, best iteration is:
[454]	valid_0's binary_logloss: 0.426085
[LightGBM] [Info] Number of positive: 1102616, number of negative: 581355
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039127 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6623
[LightGBM] [Info] Number of data points in the train set: 1683971, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654771 -> initscore=0.640079
[LightGBM] [In

In [6]:
Final_train = pd.DataFrame( {
        'LGBM1': oof_clf1.ravel(),
        'LGBM2': oof_clf2.ravel(),
        'LGBM3': oof_clf3.ravel(),
        'LGBM4' :oof_clf4.ravel(),
        'LGBM5': oof_clf5.ravel(),
    })

Final_test = pd.DataFrame( {
        'LGBM1': oof_clf1_t.ravel(),
        'LGBM2': oof_clf2_t.ravel(),
        'LGBM3': oof_clf3_t.ravel(),
        'LGBM4' :oof_clf4_t.ravel(),
        'LGBM5': oof_clf5_t.ravel(),
    })

In [7]:
param = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'learning_rate' : 0.1,}

In [8]:
kf = KFold(n_splits=5, random_state = 42,shuffle = True)

Final_oof_train = np.zeros(len(train))
Final_oof_test = np.zeros(len(test))

for (trn_idx, val_idx), test_val_idx in zip(kf.split(Final_train.values, y.values), kf.split(Final_test.values)):
        # print("fold n°{}".format(fold_))
        test_val_idx = test_val_idx[0]
        lgb_train = lgb.Dataset(Final_train.iloc[trn_idx], label = y.iloc[trn_idx])
        lgb_valid = lgb.Dataset(Final_train.iloc[val_idx], label = y.iloc[val_idx])
        
        model = lgb.train(param, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=200,verbose_eval=100)
                
        Final_oof_train[val_idx] = model.predict(Final_train.iloc[val_idx])        
        
        Final_oof_test[test_val_idx] = model.predict(Final_test.iloc[test_val_idx])
        
        # print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], Final_oof_train[val_idx])))

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1323032, number of negative: 697732
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016958 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020764, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654719 -> initscore=0.639846
[LightGBM] [Info] Start training from score 0.639846
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.866148
[200]	valid_0's auc: 0.866132
Early stopping, best iteration is:
[37]	valid_0's auc: 0.866165
AUC LGB1:0.8661645492616501 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322521, number of negative: 698244
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008243 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654466 -> initscore=0.638726
[LightGBM] [Info] Start training from score 0.638726
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.86626
[200]	valid_0's auc: 0.866238
Early stopping, best iteration is:
[47]	valid_0's auc: 0.866279
AUC LGB1:0.8662790724082267 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1323005, number of negative: 697760
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008636 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654705 -> initscore=0.639786
[LightGBM] [Info] Start training from score 0.639786
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.866286
[200]	valid_0's auc: 0.866262
Early stopping, best iteration is:
[39]	valid_0's auc: 0.866304
AUC LGB1:0.8663036226942176 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1323025, number of negative: 697740
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006687 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654715 -> initscore=0.639830
[LightGBM] [Info] Start training from score 0.639830
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.866391
[200]	valid_0's auc: 0.866354
Early stopping, best iteration is:
[61]	valid_0's auc: 0.866398
AUC LGB1:0.866398157589848 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322769, number of negative: 697996
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007747 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654588 -> initscore=0.639269
[LightGBM] [Info] Start training from score 0.639269
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.866565
[200]	valid_0's auc: 0.866537
Early stopping, best iteration is:
[48]	valid_0's auc: 0.866582
AUC LGB1:0.866582007062134 


In [13]:
Final_oof_test

744

In [14]:
def test_to_csv(preds, name:str):
    
    result = []
    for n,i in enumerate(preds):
        row = {}    
        row['id'] = n
        row['prediction'] = i
        result.append(row)
    pd.DataFrame(result).to_csv(f'output/{name}.csv', index=None)

In [16]:
from datetime import date, datetime, timezone, timedelta

KST = timezone(timedelta(hours=9))
time_record = datetime.now(KST)
_day = str(time_record)[:10]
_time = str(time_record.time())[:8]
now_time = _day+'_'+_time

test_to_csv(Final_oof_test,f'Stacking_LGBM_{now_time}')